# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
#print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("../WeatherPy/City_Data.csv")
city_data_df.head()


,Unnamed: 0,City,Country,Lat,Lon,Temp,Cloudiness(%),Windspeed,Humidity,Max Temp,Date
0,0,Khovd,MN,48.01,91.64,44.4,96,2.28,46,44.4,1586917824
1,1,Faanui,PF,-16.48,-151.75,83.3,28,5.77,65,83.3,1586917397
2,2,Saint Paul Harbor,US,57.79,-152.41,42.8,90,8.05,87,42.8,1586917825
3,3,Tasiilaq,GL,65.61,-37.64,28.4,2,6.93,63,28.4,1586917825
4,4,Barrow,US,71.29,-156.79,12.2,90,14.99,85,12.2,1586917825


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

#Create locations/humidity df's for heat layer map
locations = city_data_df[["Lat","Lon"]].astype(float)
humidity = city_data_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# My ideal weather
# humidity less than 85%
# temp > 70 and < 95
# zero cloudiness

ideal_weather_df = city_data_df[city_data_df["Cloudiness(%)"] == 0]
ideal_weather_df = ideal_weather_df[ideal_weather_df["Temp"] >= 70]
ideal_weather_df = ideal_weather_df[ideal_weather_df["Temp"] <= 95]
ideal_weather_df = ideal_weather_df[ideal_weather_df["Humidity"] <= 85]
ideal_weather_df.drop(["Unnamed: 0"], inplace=True, axis=1)
ideal_weather_df.dropna()

# output to a csv file
ideal_weather_df.to_csv("Ideal_Weather.csv")
ideal_weather_df

,City,Country,Lat,Lon,Temp,Cloudiness(%),Windspeed,Humidity,Max Temp,Date
46,Cockburn Town,TC,21.46,-71.14,78.76,0,19.01,81,78.76,1586917435
96,Ratnagiri,IN,16.98,73.30,84.11,0,7.61,66,84.11,1586917323
130,Yulara,AU,-25.24,130.99,86.00,0,6.93,13,86.00,1586917849
136,Ampanihy,MG,-24.70,44.75,73.38,0,4.99,65,73.38,1586917850
194,Puerto Colombia,CO,10.99,-74.95,80.60,0,6.93,78,80.60,1586917860
272,Beloha,MG,-25.17,45.05,73.02,0,4.99,71,73.02,1586917540
347,Bilma,NE,18.69,12.92,77.77,0,6.17,15,77.77,1586917887
378,Adrar,MR,20.50,-10.07,74.66,0,6.82,50,74.66,1586917893
390,Sulangan,PH,11.14,123.72,83.97,0,6.33,63,83.97,1586917895
429,Ransang,PH,8.96,117.58,83.91,0,3.18,61,83.91,1586917902


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_weather_df
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lon,Temp,Cloudiness(%),Windspeed,Humidity,Max Temp,Date,Hotel Name
46,Cockburn Town,TC,21.46,-71.14,78.76,0,19.01,81,78.76,1586917435,
96,Ratnagiri,IN,16.98,73.30,84.11,0,7.61,66,84.11,1586917323,
130,Yulara,AU,-25.24,130.99,86.00,0,6.93,13,86.00,1586917849,
136,Ampanihy,MG,-24.70,44.75,73.38,0,4.99,65,73.38,1586917850,
194,Puerto Colombia,CO,10.99,-74.95,80.60,0,6.93,78,80.60,1586917860,
272,Beloha,MG,-25.17,45.05,73.02,0,4.99,71,73.02,1586917540,
347,Bilma,NE,18.69,12.92,77.77,0,6.17,15,77.77,1586917887,
378,Adrar,MR,20.50,-10.07,74.66,0,6.82,50,74.66,1586917893,
390,Sulangan,PH,11.14,123.72,83.97,0,6.33,63,83.97,1586917895,
429,Ransang,PH,8.96,117.58,83.91,0,3.18,61,83.91,1586917902,


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "",  
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    lat = hotel_df["Lat"]
    lon = hotel_df["Lon"]
    city = hotel_df["City"]

    # add keyword to params dict
    params["location"] = f"{lat[index]},{lon[index]}"
    #print(params["location"])
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    #print(json.dumps(results, indent=4, sort_keys=True))
    try:
        #print(results[0]["name"])
        hotel_df["Hotel Name"][index] =results[0]["name"]
    except IndexError:
        #print("Hotel Not Found.")
        hotel_df["Hotel Name"][index] = "N/A"
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace('N/A',np.NaN)
hotel_df.dropna(inplace=True)

# output to a csv file
hotel_df.to_csv("Hotels.csv")
hotel_df

C:\Users\Joe\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Country,Lat,Lon,Temp,Cloudiness(%),Windspeed,Humidity,Max Temp,Date,Hotel Name
46,Cockburn Town,TC,21.46,-71.14,78.76,0,19.01,81,78.76,1586917435,Osprey Beach Hotel
96,Ratnagiri,IN,16.98,73.30,84.11,0,7.61,66,84.11,1586917323,Sirnaiks Ocean View
130,Yulara,AU,-25.24,130.99,86.00,0,6.93,13,86.00,1586917849,Desert Gardens Hotel - Ayers Rock Resort
136,Ampanihy,MG,-24.70,44.75,73.38,0,4.99,65,73.38,1586917850,Angora Hotel
194,Puerto Colombia,CO,10.99,-74.95,80.60,0,6.93,78,80.60,1586917860,Hotel Puerto Colombia
390,Sulangan,PH,11.14,123.72,83.97,0,6.33,63,83.97,1586917895,Maia's Beach Resort
458,Port Macquarie,AU,-31.43,152.92,78.67,0,9.80,54,79.00,1586917849,Sails Port Macquarie by Rydges
467,Loikaw,MM,19.67,97.21,90.16,0,3.98,22,90.16,1586917908,Myat Nan Taw Hotel
482,Alice Springs,AU,-23.70,133.88,87.80,0,6.93,15,87.80,1586917912,DoubleTree by Hilton Hotel Alice Springs
537,Puerto Peñasco,MX,31.32,-113.53,73.99,0,8.01,48,73.99,1586917921,Peñasco Del Sol Hotel and Conference Center


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]


In [13]:
# Add marker layer ontop of heat map
symbol_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)',
                                  stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
                                  info_box_content= [f"{hotel}" for hotel in hotel_info])

# Display figure
fig.add_layer(symbol_layer)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))